In [5]:
import matplotlib.pyplot as plt
import cirq
import numpy as np
from cirq.contrib.svg import SVGCircuit
import networkx as nx
import sympy as sp
import pandas as pd
import math

In [29]:
# Weight Block
# Crystal Problem

weight = np.array([
    [0, 1], 
    [0, 0] 
])

chem = ["Na", "Cl"]; T = len(typ) 
pos = ["0", "1"]; N = len(pos)

print(weight)

[[0 1]
 [0 0]]


In [30]:
# Define symbolic parameters
gamma = sp.Symbol("γ")
beta = sp.Symbol("β")

# Create qubits and initialize circuit
qubits = {}
for i in chem:
    for j in pos:
        name = i + j
        qubits[name] = cirq.NamedQubit(name)

print(qubits)

{'Na0': cirq.NamedQubit('Na0'), 'Na1': cirq.NamedQubit('Na1'), 'Cl0': cirq.NamedQubit('Cl0'), 'Cl1': cirq.NamedQubit('Cl1')}


In [ ]:
qaoa_circuit = cirq.Circuit()

# Initial Hadamard layer
qaoa_circuit.append([cirq.H(q) for q in qubits])

# ZZ gate definition
def cirqzz(qubits, weight, k, l, k_qu, l_qu):
    return [cirq.CNOT(qubits[k_qu], qubits[l_qu]), cirq.rz(2 * gamma * weight[k, l])(qubits[l_qu]), cirq.CNOT(qubits[k_qu], qubits[l_qu])]
            
# Cost Hamiltonian layer (ZZ interactions)
for i in range(typ_n):
    for j in range(i, typ_n):
        first_rank = i + j
        selected_weight = weight[first_rank]
        
        for k in range(pos_n):
            if selected_weight[k, k] != 0:
                qaoa_circuit.append(cirq.rz(2 * gamma * selected_weight[k, k])(qubits[k + sup_posn * first_rank]))
                
            for l in range(k+1, pos_n):
                if selected_weight[k, l] != 0:
                    qaoa_circuit.append(cirqzz(qubits, selected_weight, k, l, k + sup_posn * first_rank, l + sup_posn * first_rank))
                    

# Mixer layer (Rx rotations applied in parallel)
qaoa_circuit.append([cirq.rx(2 * beta)(q) for q in qubits])

# Measurement layer
qaoa_circuit.append([
    cirq.measure(q) for q in qubits
])

# Visualize the circuit

In [12]:
def estimate_cost(weight, samples):
    cost_value = 0.0
    node = weight.shape[0]

    # Loop over upper triangle of adjacency matrix
    for i in range(node):
        for j in range(i, node):
            if weight[i, j] != 0:
                i_samples = samples["q(" + str(i) + ")"]
                j_samples = samples["q(" + str(j) + ")"]

                # Convert {0,1} to {+1, -1}
                i_signs = (-1) ** i_samples
                j_signs = (-1) ** j_samples
                term_signs = i_signs * j_signs

                # Expectation value for edge (i,j)
                term_val = np.mean(term_signs) * weight[i, j]
                cost_value += term_val

    return cost_value

In [ ]:
import time

start = time.time()
grid_size = 11

def gam_beta_grid(weight, grid_size):
    grid = np.zeros((grid_size, grid_size))
    gamma_sweep = np.linspace(0, np.pi, grid_size)
    print(gamma_sweep)
    beta_sweep = np.linspace(0, np.pi, grid_size)
    print(beta_sweep)
    
    for i in range(grid_size):
        for j in range(grid_size):
            results = sim.sample(qaoa_circuit, params = {gamma : gamma_sweep[i], beta : beta_sweep[j]}, repetitions = 20000)
            results = dict(results)
            cost = estimate_cost(weight, results)
            grid[i, j] = cost
            
    mini, maxi = np.min(grid), np.max(grid)
    min_g, min_b = np.unravel_index(np.argmin(grid), grid.shape)
    min_gam = gamma_sweep[min_g] 
    # https://i.namu.wiki/i/f_q_IqI70RTqi_uiV4x62Y-pB--aJj6WUpDc-JMMTPL8x98fa_RVinXWilRrq6VabEc5WHLo3z3ME6APjRZDgOgHUhwPULFa3XT5P29OSMtr3saevTIeqrjORaZRqBIR1Teb9sSmhi7vB4u4ITUxGw.webp
    min_bet = beta_sweep[min_b]
    return mini, maxi, min_gam, min_bet, grid
        
mini, maxi, min_gam, min_bet, grid = gam_beta_grid(weight, grid_size)
print(mini, maxi, min_gam, min_bet)


end = time.time()

print(end - start)